In [10]:
# Created by Balakrishna Vagvala and Sherrill Kirk
# This code depends upon the packages and libraries listed in the next cell
# Date of creation: 12/1/2020
# Script purpose: this script processes the OSU CCAL Lab delivered data, compares with expected fields file and make ready to be uploaded to database.

In [11]:
#importing pandas for data operations, numpy for numerical operations. time is used to have present time.
import pandas as pd
import numpy as np
import time
#sequence matcher is used to compare strings and openpyxl is used to write workbooks.
from difflib import SequenceMatcher
from openpyxl import load_workbook
from shutil import copyfile
#cd to change current directory where output goes. this has to be manually entered
%cd D:\ROMN\working\Misc\4People\4Bala\WaterQualityProcessing\CCAL\2020\Preproccessed2

D:\ROMN\working\Misc\4People\4Bala\WaterQualityProcessing\CCAL\2020\Preproccessed2


In [12]:
# mounting google drive for google collab, not needed for native IDE's
# from google.colab import drive
# drive.mount('/content/drive')

In [13]:
#importing reading functions and declaring the location of our file.
import os, glob
#file to check crosswalks, i.e field names, this is an excel file's path
crosswalk = r'D:\ROMN\working\Misc\4People\4Bala\WaterQualityProcessing\CCAL\OSU_CCAL_ExpectedFields_CrossWalk_20201214.xlsx'
#toread crosswalk file
crosswalk = pd.read_excel(crosswalk)
#file to process , this is an excel file's path
data = r'D:\ROMN\working\Misc\4People\4Bala\WaterQualityProcessing\CCAL\2020\GRKO_032620_changed_names.xlsx'


currentoutputpath= r'D:\ROMN\working\Misc\4People\4Bala\WaterQualityProcessing\CCAL\2020\Preproccessed2'

filename= os.path.basename(data)
print(filename)

firstline=[]
firstline.append("File Being Processed is: " + filename+ "\n \n \n#########\n\n" )
#toread the lab file, we mention sheet name of excel file in the second argument, here GRKO Data is our sheet name in excel file by lab
# osudata = pd.read_excel(data, 'GRKO Data')
#to specify sheet number instead of sheet name
osudata = pd.read_excel(data, sheet_name = 1)

GRKO_032620_changed_names.xlsx


In [14]:
#here we process our data so that we have just the field names and field values in our dataframe
firstline.append("File " + filename+ " is Processed \n \n \n#########\n" )
df = osudata
df = df.iloc[2:]
df = df.rename(columns=df.iloc[0])
df.reset_index(inplace=True)
df = df.iloc[1:,1:]
display(df)

,SampleName,Project Code,Lab Number,Site ID,Remark,Delivery Date,Alkalinity\n(mg),Date,DOC (mg C/L),Date,...,SO4-S\r\n(mg S/L),Date,NO3\n(mg/L),Date,Na\n(mg/L),Date,Duplicate NO3\r\n(mg/L),Date,Cl (mg/L) Cl(mg/L),pqrstuvwxyz
1,GRKO101,GRKO,101,19NPS0000602,GRKO_S01 3/24/20 13:45,3/26/2020,134.90,4/28/2020,12.50,4/14/2020,...,34.91,5/13/2020,17.07,6/2/2020,0.81,4/29/2020,0.82,4/29/2020,NaN,NaN


In [15]:
# to remove \n and \r from lab delivareables, they are influencing our data and output, so we are removing them

dfStrip = osudata.replace('\n',' ', regex=True)

dfStrip2 = dfStrip.replace('\r','', regex=True)

df = dfStrip2

df = df.iloc[2:]

df = df.rename(columns=df.iloc[0])

df.reset_index(inplace=True)

df = df.iloc[1:,1:]

display(df)

,SampleName,Project Code,Lab Number,Site ID,Remark,Delivery Date,Alkalinity (mg),Date,DOC (mg C/L),Date,...,SO4-S (mg S/L),Date,NO3 (mg/L),Date,Na (mg/L),Date,Duplicate NO3 (mg/L),Date,Cl (mg/L) Cl(mg/L),pqrstuvwxyz
1,GRKO101,GRKO,101,19NPS0000602,GRKO_S01 3/24/20 13:45,3/26/2020,134.90,4/28/2020,12.50,4/14/2020,...,34.91,5/13/2020,17.07,6/2/2020,0.81,4/29/2020,0.82,4/29/2020,NaN,NaN


In [16]:
osufields = list(df.columns)
labfields = crosswalk['NativeDeliveryFIelds'].tolist()
expectedfields = crosswalk['CrossWalkedFields'].tolist()
timestr = time.strftime("_%y_%m_%d_%H_%M")
# this cell is to check for coloumns and export irregularities into warning list
f= open("warning_list of "+filename+" at "+timestr+".txt","w+")
matchingfields =[]
morethanninety = []
lessthanninety = []
newfields=[]
duplicatefields=[]
missedfields = []
p=''
q=''
x=''
duplicatefields.append ('\n\n#####\n\nFields with duplicate field names \n') 
missedfields.append ('\n\n######\n\nFields which are expected, but not in our data\n')
matchingfields.append ('\n\n#####\n\nFields that matched as expected') 
morethanninety.append ('\n\n#####\n\nFields without 100% match but had more than 90% Match\n') 
lessthanninety.append ('\n\n#####\n\nFields without 100% match but had less than 90% Match\n') 
newfields.append ('\n\n#####\n\nNew Fields , these are not found in crosswalk\n') 



In [17]:
for i in osufields:
    if i == 'Date': 
        x = osufields.index(i)
        osufields[x] = osufields[x-1] +(" ") +osufields[x]
        continue 
for j in labfields:
    if j == 'Date':
        x = labfields.index(j)
        labfields[x] = labfields[x-1] +(" ") + labfields[x]
df.columns = osufields


In [18]:
if len(osufields)>len(labfields):
    matchingfields.append('This '+filename+' file has new columns, is a high level threat, look into it.\n')

if len(osufields)<len(labfields):
    matchingfields.append('This '+filename+' file has lesser columns than expected, is a high level threat, look into it.\n')

In [19]:
for i in osufields:   
    for j in labfields:
        if j != 'Date' and i != 'Date' :
            if 'Date' not in i:
                if j in i:                
                    ratio = SequenceMatcher(None,i,j).ratio()
                    if ratio != 1:

                        duplicatefields.append("Warning: "+i+ " is a duplicate field, almost matched with "+i +" look into it\n")                    

                        x = list(df.columns).index(i)
                        cols = [i for i in range(df.shape[1])]
                        cols.remove(x)
                        cols.remove(x+1)
                        df = df.iloc[:,cols]
                        
                        z = x+1
                        del osufields[x]
                        del osufields[x]


In [20]:
for i in osufields:
    if i not in labfields:
        newfields.append("Warning: "+i+" is a new field, look into it\n")        

In [21]:

for i in osufields:     
    if i in labfields:
        matchingfields.append("coloumn: "+ i +"  has sucessfully matched \n")
        x = osufields.index(i)
        y = labfields.index(i)        
        osufields[x] = expectedfields[y]
        continue      

    else:
        result = ""
        count = 0
        ratio = 0  
        
        for words in labfields:
            ratio = SequenceMatcher(None,i,words).ratio()
            if ratio > count:
                count = ratio
                result =words         

        if count > 0.9:
            x = osufields.index(i)
            y = labfields.index(result)        
            morethanninety.append("Warning: "+osufields[x]+" was crosswalk to "+ result + " has matched with more than 90% accuracy and may have an error, look into it \n")
            osufields[x] = expectedfields[y]


        elif count> 0.8:
            x = osufields.index(i)
            y = labfields.index(result)
            lessthanninety.append("Warning: "+osufields[x]+" was crosswalk to "+ result + " has matched with more than 90% accuracy and may have an error, look into it \n")
            osufields[x] = expectedfields[y]

            
            
# temp to check the missing columns in our processed lab deliverable 
missedfields = []
missedfields.append ('\n\n######\n\nFields which are expected, but not in our data\n')
temp =  expectedfields.copy()
for i in osufields:
    if i in temp:
        temp.remove(i) 
for i in temp:
    missedfields.append("column: "+ i + " is expected but is not in our current data \n ")        
worked_files = firstline + matchingfields + morethanninety +lessthanninety + duplicatefields + missedfields+newfields
for i in range(len(worked_files)):
    f.write(worked_files[i])
        
f.write('\n\n######\nthis is end of script')
f.close()
df.columns = osufields

In [22]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# df

In [23]:
#saving our new csv file into a downloadable format. and it can be saved
# df.to_csv( "data_fieldsprocessed"+timestr+".csv", index=False, encoding='utf-8-sig')
#if you want an excel output
# f = open
# df.to_excel("pre-processed"+timestr+".xls", index=False, encoding='utf-8-sig')
# runtime for file
runtimenow = timestr

In [24]:
# to make a duplicate of our file and export our processed 
# copyfile ('D:\Bala\OSU CCAL\input\GRKO_032620.xlsx' , './processed'+filename+'.xlsx')
processed_path = (currentoutputpath+'\processed'+filename)
copyfile (data , processed_path)
data

'D:\\ROMN\\working\\Misc\\4People\\4Bala\\WaterQualityProcessing\\CCAL\\2020\\GRKO_032620_changed_names.xlsx'

In [25]:
# rename our copied file and place it at the location we want

# os.rename(r'D:\Bala\OSU CCAL\output\processed.xlsx', r'D:\Bala\OSU CCAL\output\OSUCCAL_processed_' + filename + '.xlsx')

In [26]:
# enter the path of the processed named copy of the original file
# processed_path = 'D:\Bala\OSU CCAL\output\OSUCCAL_processed__filename.xlsx'
# to load our data frame into excel sheet
book = load_workbook(processed_path)
writer = pd.ExcelWriter(processed_path, engine = 'openpyxl')
writer.book = book
# enter the desired sheet name below
sheetname = 'enter here ' + 'preprocessed' #enter your sheet name here in the space, we can have it as a prefix as you want
df.to_excel(writer, sheet_name = sheetname)
writer.save()
writer.close()

In [27]:
# val = input("Do you want to merge the processed sheets into a single sheet,Enter 1 for yes, 0 for no: ") 

# listofdataframes = []
# path = input("enter the path of folder having file")
# all_files = glob.glob(os.path.join(path, "GRKO*.xlsx"))
# for  no,f in enumerate(all_files):
#   df = pd.read_excel(f, sheet_name = 2)
#   listofdataframes.append(df)
# combined_file = pd.concat( listofdataframes) 
# timestr = time.strftime("_%m_%d_%Y_%H_%M") 
# combined_file.to_csv( "combined_csv"+timestr+".csv", index=False, encoding='utf-8-sig')
